# Template Notebook for NSGA II Multi-Objective-Optimization exploring the Prompt Embedding space
Notebook Version: 0.3 (06/03/2024)
* added Google Colab support

## Google Colab Setup

In [ ]:
# Google Colab: Execute this to install packages and setup drive
!pip install "evolutionary[all] @ git+https://git@github.com/malthee/evolutionary-diffusion.git"

In [ ]:
# Mount drive to save results
from google.colab import drive
import evolutionary_imaging.processing as ip
drive.mount("/content/drive")
base_path = "/content/drive/MyDrive/evolutionary/"
ip.RESULTS_FOLDER = base_path + ip.RESULTS_FOLDER

In [ ]:
# Check if GPU is available
import torch
print(torch.cuda.is_available())

## Project Setup

In [ ]:
from evolutionary.plotting import plot_fitness_statistics
import evolutionary_imaging.processing as ip
from diffusers.utils import logging
from evolutionary_imaging.processing import create_animation_from_generations, create_generation_image_grid, save_images_from_generation
import torch
import os

In [ ]:
logging.disable_progress_bar() # Or else your output will be full of progress bars
logging.set_verbosity_error() # Enable again if you are having problems
os.environ["TOKENIZERS_PARALLELISM"] = "false" # To remove warning of libraries using tokenizers
# Change the results folder for images if you want to 
# ip.RESULTS_FOLDER = 'choose_your_destination'

def save_images_post_evaluation(g, a):
    save_images_from_generation(a.population, g)
    
# Check torch random state, used across all libraries. Caution setting fixed seeds as it affects not only generation but also variation.
print(torch.random.get_rng_state())

In [ ]:
from evolutionary_prompt_embedding.image_creation import SDXLPromptEmbeddingImageCreator
from evolutionary_prompt_embedding.variation import PooledArithmeticCrossover, PooledUniformGaussianMutator, UniformGaussianMutatorArguments
from evolutionary.evolutionary_selectors import TournamentSelector
from evolutionary.algorithms.nsga_ii import NSGA_II
from evolutionary_imaging.evaluators import AestheticsImageEvaluator, CLIPScoreEvaluator, MultiCLIPIQAEvaluator, \
    SingleCLIPIQAEvaluator
from evolutionary.evolution_base import MultiObjectiveEvaluator

prompt = "van gogh"
population_size = 5
num_generations = 5
batch_size = 2
elitism = 1

creator = SDXLPromptEmbeddingImageCreator(batch_size=batch_size, inference_steps=3)
evaluator = MultiObjectiveEvaluator([
    AestheticsImageEvaluator(), 
    CLIPScoreEvaluator(prompt=prompt),
    MultiCLIPIQAEvaluator(metrics=('quality', 'brightness', ('good', 'bad'))),
    SingleCLIPIQAEvaluator(metric=('testness', 'notntestness'))
])
crossover = PooledArithmeticCrossover(crossover_rate=0.5, crossover_rate_pooled=0.5)
# clamp_range was evaluated with pre-testing/clamp_range/sdxl_turbo.py
mutation_arguments = UniformGaussianMutatorArguments(mutation_rate=0.1, mutation_strength=2.5, clamp_range=(-900, 900)) 
mutation_arguments_pooled = UniformGaussianMutatorArguments(mutation_rate=0.1, mutation_strength=0.5, clamp_range=(-8, 8))
mutator = PooledUniformGaussianMutator(mutation_arguments, mutation_arguments_pooled)
selector = TournamentSelector(tournament_size=3)

# Prepare initial arguments
init_embed = creator.arguments_from_prompt(prompt)
init_args = [init_embed for _ in range(population_size)]

nsga = NSGA_II(
    num_generations=num_generations,
    population_size=population_size,
    solution_creator=creator,
    selector=selector,
    crossover=crossover,
    mutator=mutator,
    evaluator=evaluator,
    elitism_count=elitism,
    initial_arguments=init_args,
    post_evaluation_callback=lambda g, a: save_images_from_generation(a.population, g)  # TODO save only best images (in front 0) post sorting
)

In [ ]:
best_solution = nsga.run()

In [ ]:
from diffusers.utils import make_image_grid

# Show best solution
print(best_solution.fitness)
make_image_grid(best_solution.result.images, 1, batch_size)

## Visualize the evolution

In [ ]:
for gen in range(num_generations):
    create_generation_image_grid(gen, max_images=10)

In [ ]:
video_loc = create_animation_from_generations(num_generations)
print(video_loc)

## Plot fitness statistics

In [ ]:
plot_fitness_statistics(num_generations, nsga.best_fitness, nsga.worst_fitness, nsga.avg_fitness, title="Aesthetics", multi_objective_plot_index=0)

In [ ]:
plot_fitness_statistics(num_generations, nsga.best_fitness, nsga.worst_fitness, nsga.avg_fitness, title="CLIP Score", multi_objective_plot_index=1)

## Save notebook and components

In [ ]:
!jupyter nbconvert --to html nsga_notebook.ipynb

### Save the run to disk

In [ ]:
import pickle
import os
from datetime import datetime

os.makedirs("saved_runs", exist_ok=True)
with open(os.path.join("saved_runs", f"nsga_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl"), "wb") as f:
    pickle.dump(nsga, f)

### Load the run from disk 
Notebook and library versions should match with the saved run

In [ ]:
import pickle
import os

with open(os.path.join("saved_runs", "insert_filename"), "rb") as f:
    run = pickle.load(f)

## Fallback functions for when something went wrong

### Access Best Solution from Disk

In [ ]:
import os
import glob
import evolutionary_imaging.processing as ip
from PIL import Image

num_generations = 4  # Set this to the number of generations you ran (if you didn't finish)
generation_dir = os.path.join("results", f"{num_generations}")
image_files = glob.glob(os.path.join(generation_dir, "*.png"))
image_files.sort(key=ip.fitness_filename_sorting_key, reverse=True)
print(image_files[0])
Image.open(image_files[0])

### ffmpeg is not installed, create GIF instead

In [ ]:
from evolutionary_imaging.processing import create_animation_from_generations_pil
video_loc = create_animation_from_generations_pil(num_generations)
print(video_loc)